## Paper LBO Model Example

When interviewing for a junior private equity position, a candidate must prepare for in-office modeling tests 
on potential private equity investment opportunities—especially LBO scenarios. Hence, the model serves to help 
those candidates preparing for an interview or planning to join PE World. 

In [ ]:
Source: http://www.streetofwalls.com/finance-training-courses/private-equity-training/paper-lbo-model-example/

## Parameters and Assumptions to build the LBO Model

XYZ Private Equity Partners purchases ABC Target Company for 5.0x Forward 12 months (FTM) EBITDA at the end of Year 0.

The debt-to-equity ratio for the LBO acquisition will be 60:40.

Assume the weighted average interest rate on debt to be 10%.

ABC expects to reach $100 million in sales revenue with an EBITDA margin of 40% in Year 1.

Revenue is expected to increase by 10% year-over-year (y-o-y).

EBITDA margins are expected to remain flat during the term of the investment.

Capital expenditures are expected to equal 15% of sales each year.

Operating working capital is expected to increase by $5 million each year.

Depreciation is expected to equal $20 million each year.

Assume a constant tax rate of 40%.

XYZ exits the target investment after Year 5 at the same EBITDA multiple used at entry (5.0x FTM EBITDA).

Assume all debt pay-down occurs at the moment of sale at the end of Year 5 (this eliminates the iterative/circular dependency between debt pay-down/cash balances and interest expense in a computer-based LBO model).

By using the instructions we have created a dictionary with Entry Assumptions using the information provided

### 1. CALCULATE THE PURCHASE PRICE OF ABC

    Using a 5.0x entry multiple, calculate the price paid by multiplying by Year 1 EBITDA. 
    $40 million in EBITDA (which represents a 40% EBITDA margin on $100 million in revenue) multiplied by 5.

In [1]:
# import libs
import pandas as pd
import numpy as np

In [2]:
entry_ass = {
    "entry_multiple":5,
    "Rev_Y1":100,
    "EBITDA_Y1_Margin":.4,
    "debt_ratio": .6,
    "equity_ratio": .4,
}
entry_ass['EBITDA_Y1']=entry_ass['Rev_Y1']*entry_ass['EBITDA_Y1_Margin']
entry_ass["price_paid"]=entry_ass["entry_multiple"]*entry_ass["EBITDA_Y1"]
entry_ass

{'entry_multiple': 5,
 'Rev_Y1': 100,
 'EBITDA_Y1_Margin': 0.4,
 'debt_ratio': 0.6,
 'equity_ratio': 0.4,
 'EBITDA_Y1': 40.0,
 'price_paid': 200.0}

### 2. CALCULATE THE DEBT AND EQUITY FUNDING AMOUNTS USED FOR THE PURCHASE PRICE.

    The given information assumes debt to equity ratio of 60:40 for the purchase price.
    Debt portion = 60% × $200 million, or $120 million.
    Equity portion = 40% × $200 million, or $80 million.

In [3]:
# getting the Equity and Debt portion of the total price amount

entry_ass["debt_portion"]=entry_ass['debt_ratio']*entry_ass['price_paid']
entry_ass['equity_portion']=entry_ass['equity_ratio']*entry_ass['price_paid']
entry_ass

{'entry_multiple': 5,
 'Rev_Y1': 100,
 'EBITDA_Y1_Margin': 0.4,
 'debt_ratio': 0.6,
 'equity_ratio': 0.4,
 'EBITDA_Y1': 40.0,
 'price_paid': 200.0,
 'debt_portion': 120.0,
 'equity_portion': 80.0}

### 3. BUILD THE INCOME STATEMENT

     (Notice that, because the exit value at the end of Year 5 will be based on a forward EBITDA multiple,
      we must calculate six year’s worth of income statement, not 5. Also note that the numbers might not 
      agree perfectly because of rounding. It is reasonable to round your intermediate calculations to the 
      nearest integer in carrying over calculations to the next step.)

In [4]:
# Create the DataFrame for the income statement: All figures are in Millionss
years=[1,2,3,4,5,6]
rows=["Rev","EBITDA","less: D&A","EBIT", "less: Interest","EBT", "less: Taxes","Earnings"]
lbo_is = pd.DataFrame(data=None,index=rows,columns=years)
lbo_is

,1,2,3,4,5,6
Rev,NaN,NaN,NaN,NaN,NaN,NaN
EBITDA,NaN,NaN,NaN,NaN,NaN,NaN
less: D&A,NaN,NaN,NaN,NaN,NaN,NaN
EBIT,NaN,NaN,NaN,NaN,NaN,NaN
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
EBT,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Earnings,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Create a dictionary for Income Statement assumptions
is_ass={
    "rev_growth": 1.1,
    "int_rate": .10,
    "EBITDA_margin": entry_ass['EBITDA_Y1_Margin'],
    'tax_rate':.4,
    'capex % of rev':.15,
    'change_in_NWC':5,
}
is_ass

{'rev_growth': 1.1,
 'int_rate': 0.1,
 'EBITDA_margin': 0.4,
 'tax_rate': 0.4,
 'capex % of rev': 0.15,
 'change_in_NWC': 5}

In [6]:
#Start with building out Revenue, assuming a 10% growth rate we will use a While loop to forecast

lbo_is[1].loc['Rev']=entry_ass['Rev_Y1']
i = 2
while i <= len(lbo_is.loc['Rev']):
    lbo_is[i].loc['Rev']=lbo_is[(i-1)].loc['Rev']*1.10
    i+=1
    
lbo_is

,1,2,3,4,5,6
Rev,100,110.0,121.0,133.1,146.41,161.051
EBITDA,NaN,NaN,NaN,NaN,NaN,NaN
less: D&A,NaN,NaN,NaN,NaN,NaN,NaN
EBIT,NaN,NaN,NaN,NaN,NaN,NaN
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
EBT,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Earnings,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Build out EBITDA using % EBITDA margin assumptions
lbo_is.loc['EBITDA']= lbo_is.loc['Rev']*is_ass['EBITDA_margin']
lbo_is 

,1,2,3,4,5,6
Rev,100,110.0,121.0,133.1,146.41,161.051
EBITDA,40.0,44.0,48.4,53.24,58.564,64.4204
less: D&A,NaN,NaN,NaN,NaN,NaN,NaN
EBIT,NaN,NaN,NaN,NaN,NaN,NaN
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
EBT,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Earnings,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Setting Depreciation & Amortization
lbo_is.loc['less: D&A']=-20
lbo_is 

,1,2,3,4,5,6
Rev,100,110.0,121.0,133.1,146.41,161.051
EBITDA,40.0,44.0,48.4,53.24,58.564,64.4204
less: D&A,-20,-20,-20,-20,-20,-20
EBIT,NaN,NaN,NaN,NaN,NaN,NaN
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
EBT,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Earnings,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Subtract Depreciation & Amortization (D&A) to get EBIT.
lbo_is.loc['EBIT']=lbo_is.loc['EBITDA']+lbo_is.loc['less: D&A']
lbo_is

,1,2,3,4,5,6
Rev,100,110.0,121.0,133.1,146.41,161.051
EBITDA,40.0,44.0,48.4,53.24,58.564,64.4204
less: D&A,-20,-20,-20,-20,-20,-20
EBIT,20.0,24.0,28.4,33.24,38.564,44.4204
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
EBT,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Earnings,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Calculate interest expense using the debt amount portion used for purchase multiplied 
# by the interest rate to calculate the yearly interest expense line item.

lbo_is.loc['less: Interest']=-is_ass['int_rate']*entry_ass['debt_portion']
lbo_is

,1,2,3,4,5,6
Rev,100,110.0,121.0,133.1,146.41,161.051
EBITDA,40.0,44.0,48.4,53.24,58.564,64.4204
less: D&A,-20,-20,-20,-20,-20,-20
EBIT,20.0,24.0,28.4,33.24,38.564,44.4204
less: Interest,-12.0,-12.0,-12.0,-12.0,-12.0,-12.0
EBT,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Earnings,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Calculate Earnings Before Tax (EBT).

lbo_is.loc['EBT']=lbo_is.loc['EBIT']+lbo_is.loc['less: Interest']
lbo_is

,1,2,3,4,5,6
Rev,100,110.0,121.0,133.1,146.41,161.051
EBITDA,40.0,44.0,48.4,53.24,58.564,64.4204
less: D&A,-20,-20,-20,-20,-20,-20
EBIT,20.0,24.0,28.4,33.24,38.564,44.4204
less: Interest,-12.0,-12.0,-12.0,-12.0,-12.0,-12.0
EBT,8.0,12.0,16.4,21.24,26.564,32.4204
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Earnings,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Subtract taxes using the tax rate to get to tax-effected EBT (a proxy for Net Income)

lbo_is.loc['less: Taxes']=lbo_is.loc['EBT']*-is_ass['tax_rate']
lbo_is

,1,2,3,4,5,6
Rev,100,110.0,121.0,133.1,146.41,161.051
EBITDA,40.0,44.0,48.4,53.24,58.564,64.4204
less: D&A,-20,-20,-20,-20,-20,-20
EBIT,20.0,24.0,28.4,33.24,38.564,44.4204
less: Interest,-12.0,-12.0,-12.0,-12.0,-12.0,-12.0
EBT,8.0,12.0,16.4,21.24,26.564,32.4204
less: Taxes,-3.2,-4.8,-6.56,-8.496,-10.6256,-12.96816
Earnings,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
lbo_is.loc['Earnings']=lbo_is.loc['EBT']+lbo_is.loc['less: Taxes']
lbo_is

,1,2,3,4,5,6
Rev,100,110.0,121.0,133.1,146.41,161.051
EBITDA,40.0,44.0,48.4,53.24,58.564,64.4204
less: D&A,-20,-20,-20,-20,-20,-20
EBIT,20.0,24.0,28.4,33.24,38.564,44.4204
less: Interest,-12.0,-12.0,-12.0,-12.0,-12.0,-12.0
EBT,8.0,12.0,16.4,21.24,26.564,32.4204
less: Taxes,-3.2,-4.8,-6.56,-8.496,-10.6256,-12.96816
Earnings,4.8,7.2,9.84,12.744,15.9384,19.45224


### 4. CALCULATE CUMULATIVE LEVERED FREE CASH FLOW (FCF)

In [14]:
# Create the DataFrame for the Free Cash Flow

lbo_fcf=pd.DataFrame(data=None, index=["Earnings", "plus: D&A", "less: capex", "less: NWC", "FCF"], columns=years)
lbo_fcf

,1,2,3,4,5,6
Earnings,NaN,NaN,NaN,NaN,NaN,NaN
plus: D&A,NaN,NaN,NaN,NaN,NaN,NaN
less: capex,NaN,NaN,NaN,NaN,NaN,NaN
less: NWC,NaN,NaN,NaN,NaN,NaN,NaN
FCF,NaN,NaN,NaN,NaN,NaN,NaN


#### Preparing Free Cash Flow
    
    (NOTE: We do not need Year 6 capital expenditures, or Free Cash Flow for that matter, 
     because EBITDA does not incorporate capex and because only FCF in Years 1-5 can be used to pay down debt.)
    
     Calcs step:

     1. Start with EBT (Tax-effected) and then add back non-cash expenses (D&A).
     2. Add D&A to EBT
     3. Subtract capital expenditures (Capex).
     4. Subtract the annual increase in operating working capital to get to Free Cash Flow (FCF).
     5. Calculate Cumulative Free Cash Flow during the life of the LBO by adding FCF year-to-year

In [15]:
lbo_fcf.loc['Earnings']=lbo_is.loc['Earnings']
lbo_fcf.loc['plus: D&A']=-lbo_is.loc['less: D&A']
lbo_fcf.loc['less: capex']=lbo_is.loc['Rev']*-is_ass['capex % of rev']
lbo_fcf.loc['less: NWC']=-is_ass['change_in_NWC']
lbo_fcf.loc['FCF']=lbo_fcf.loc[['Earnings','plus: D&A','less: capex', 'less: NWC']].sum()
lbo_fcf

,1,2,3,4,5,6
Earnings,4.8,7.2,9.84,12.744,15.9384,19.45224
plus: D&A,20,20,20,20,20,20
less: capex,-15.0,-16.5,-18.15,-19.965,-21.9615,-24.15765
less: NWC,-5,-5,-5,-5,-5,-5
FCF,4.8,5.7,6.69,7.779,8.9769,10.29459


In [16]:
# Calculate Cumulative Free Cash Flow during the life of the LBO by adding FCF year-to-year

cumulative_fcf=lbo_fcf[[1,2,3,4,5]].loc['FCF'].sum()
cumulative_fcf

33.94590000000002

### CALCULATE ENDING PURCHASE PRICE (EXIT VALUE) AND RETURNS

  Cals steps:
        
        1. Calculate Total Enterprise Value (TEV) at Exit: Take Forward EBITDA at exit (Year 6 EBITDA) along 
           with a 5.0x exit multiple to calculate Exit TEV.
        2. Calculate Net Debt at Exit (also known as Ending Debt): 
           Beginning Debt – Debt Pay-down = Ending Debt.
           Beginning Debt – Cumulative FCF = Ending Debt
            
        3. Calculate ending Equity Value (EV) by subtracting Ending Debt from Exit TEV.
        4. Calculate the Multiple-of-Money (MoM) EV return (Ending EV ÷ Beginning EV).

In [17]:
exit_returns={
    "exit_EBITDA":lbo_is[6].loc['EBITDA'],
    'exit_multiple':entry_ass['entry_multiple'],
    'beginning_debt':entry_ass['debt_portion'],
    'cumulative_FCF':cumulative_fcf,
}
exit_returns

{'exit_EBITDA': 64.42040000000004,
 'exit_multiple': 5,
 'beginning_debt': 120.0,
 'cumulative_FCF': 33.94590000000002}

In [18]:
exit_returns['exit_TEV']= exit_returns['exit_EBITDA']*exit_returns['exit_multiple']
exit_returns['ending_debt']= exit_returns['beginning_debt']-exit_returns['cumulative_FCF']
exit_returns['ending_equity']=exit_returns['exit_TEV']-exit_returns['ending_debt']
exit_returns['MoM']=exit_returns['ending_equity']/entry_ass['equity_portion']
exit_returns

{'exit_EBITDA': 64.42040000000004,
 'exit_multiple': 5,
 'beginning_debt': 120.0,
 'cumulative_FCF': 33.94590000000002,
 'exit_TEV': 322.1020000000002,
 'ending_debt': 86.05409999999998,
 'ending_equity': 236.04790000000023,
 'MoM': 2.950598750000003}

### Estimate IRR based on the MoM multiple.

    The following table is useful for estimating IRR based upon 5-year MoM multiples:
    2.0x MoM over 5 years ~15% IRR
    2.5x MoM over 5 years ~20% IRR
    3.0x MoM over 5 years ~25% IRR
    3.7x MoM over 5 years ~30% IRR
    
    Therefore, we can assume that the implied IRR for the paper LBO case study is approximately 25%, or slightly below